## This is to clean the data that I have acquired

FIRST THING TO NOTE

I sorted the pdfs and csv manually and that allowed me to arrange them by year. This allows me to sort them by a sort of univesal format for each folder.

The problem with the current data is that it's a few csvs, with the first few lines, I am too scared to alter the csvs, so let's just append all of them and clean within the DF.

First, let's skip the first rows of every CSV and the line which repeats itself. The below commands also clean it up at the individual file level and and the ends of each csv. 

In [1]:
import os
import pandas as pd

#First, we read the whole folder, which has all the data
csv_path = "CSVs/2021"
files = []
for file in os.listdir(csv_path):
    if (file != ".DS_Store"):
        files.append(os.path.join(csv_path, file))
files
#Merge it
dfs = []
for file in files:
    df = pd.read_csv(file)

#Clean up at the merge and page breaks. 

    if(df.iloc[0][1] != "Name of Indian Company"):
        df = df.iloc[1:]
    df.columns = df.iloc[0]

#Skipping the titles row.

    df = df[1:-1]
    df.iloc[0, 0] = "01"
    dfs.append(df)

## Editorial Decisions

Over here, I merged the data frames, in retrospect, i would have thought about it a bit more, but I guess this will serve for now. I used the Number column (#) to clean up as well because that seemed to be a good metric to order things by.

I also made the decision to drop quarters and add years instead. I also noticed later that because of the conversion, a lot of the dollar values slipped into zeo, so that's something to keep in mind.

In [2]:
merged = pd.concat(dfs)
merged = merged.loc[:, merged.columns.notnull()]
merged.columns = ["#", "Company", "State", "District", "Item", "Foreign Collaborator", "Country", "Rupees", "Dollars", "Type", 'Rup2', 'Dol2', 'RupC', 'Dol3']
merged = merged.reset_index(drop=True)
merged.drop(merged[merged['Company'] == 'Name of Indian Company'].index, inplace = True)   
merged = merged.reset_index(drop=True)
merged

,#,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Rup2,Dol2,RupC,Dol3
0,01,3PLANES SPORTSTECH PRIVATE LIMITED,MAHARASHTRA,Mumbai,Providing software support and maintenance to ...,SURGE SPORTSTECH INC.,U.S.A,0.02,0,Received in the Company,NaN,NaN,NaN,NaN
1,2,3PLANES SPORTSTECH PRIVATE LIMITED,MAHARASHTRA,Mumbai,Providing software support and maintenance to ...,SURGE SPORTSTECH INC.,U.S.A,0.02,0,Received\nin the Company,NaN,NaN,NaN,NaN
2,3,3Y3 DIGITAL LABS PRIVATE LIMITED,KARNATAKA,Bengaluru,Other information service activities n.e.c.,"AUTOMATAD ,INC",U.S.A,5,0.07,Received in the Company,NaN,NaN,NaN,NaN
3,4,A.S. Carehomes Private Limited,KERALA,Thiruvananthapuram,Residential care activities for the elderly an...,Sajan Janardhanan Pillai,U.S.A,8.86,0.12,Received in the\nCompany,NaN,NaN,NaN,NaN
4,5,AAG CENTRE FOR AVIATION TRAINING PRIVATE LIMITED,TAMIL NADU,Chennai,FLYING SCHOOL,Alpha Aviation Academy (Europe) Limited,United Kingdom,0.01,0,Received in the Company,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Company,NaN,NaN,NaN,NaN
13087,44,Trent Hypermarket Private Limited,MAHARASHTRA,Mumbai,Retail sale in non-specialized stores with foo...,TESCO Overseas Investments Limited,United Kingdom,NaN,NaN,Received in the\nCompany,400.03,5.46,NaN,NaN
13088,45,WACOAL INDIA PRIVATE LIMITED,MAHARASHTRA,Mumbai,"Retail sale of readymade garments, hosiery goo...",Wacoal International Hong Kong Company Limited,HongKong,NaN,NaN,Received in the Company,38.25,0.52,NaN,NaN
13089,46,Z-WEAR DISTRIBUTION\nINDIA PRIVATE LIMITED,MAHARASHTRA,Mumbai,Other non-specialised wholesale trade n.e.c.,Future Business Concepts Limited,HongKong,NaN,NaN,Received in\nthe Company,2.4,0.03,NaN,NaN


In [3]:
nans = merged[(merged["#"].isna())]

for i in range(len(merged)) :
#     print(merged.loc[i, "#"])
#     print(merged.loc[i + 1, "#"])
    if(isinstance(merged.loc[i, "#"], str) and len(merged.loc[i, "#"]) == 3):
        if(isinstance(merged.loc[i + 1, "#"], str) and len(merged.loc[i + 1, "#"]) == 1):
            nans = nans.append(merged.loc[i+1])

inc = nans[nans["Company"].notna()].index

edited = merged.drop(merged[(merged["#"].isna()) & (merged['Company'].isna())].index)
edited = edited.drop(edited[edited['Company'].isna()].index)
for num in inc:
    edited.loc[num-1, 'Company'] =  merged.loc[num - 1, 'Company'] + " " + merged.loc[num, 'Company']
    edited.drop(labels=num, axis=0, inplace=True)   
edited = edited.reset_index(drop=True)

for i in edited.index:
    edited.loc[i, 'State'] = str(edited.loc[i, 'State']).replace('\n',' ')
    edited.loc[i, 'Foreign Collaborator'] = str(edited.loc[i, 'Foreign Collaborator']).replace('\n',' ')
    edited.loc[i, 'Company'] = str(edited.loc[i, 'Company']).replace('\n',' ')
    edited.loc[i, 'Type'] = str(edited.loc[i, 'Type']).replace('\n',' ')

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_18357/2011025732.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans = nans.append(merged.loc[i+1])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_18357/2011025732.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans = nans.append(merged.loc[i+1])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_18357/2011025732.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans = nans.append(merged.loc[i+1])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_18357/2011025732.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans = nans.append(merge

In [4]:
import math

nalist = edited

#This is more cleanup and conversions to numeric for us to cleanup.

nalist['Rupees'] = pd.to_numeric(nalist['Rupees'].str.replace(",", ""))
nalist['Dollars'] = pd.to_numeric(nalist['Dollars'].str.replace(",", ""))
nalist['Dol2'] = pd.to_numeric(nalist['Dol2'].str.replace(",", ""))
nalist['Dol3'] = pd.to_numeric(nalist['Dol3'].str.replace(",", ""))
nalist['Rup2'] = pd.to_numeric(nalist['Rup2'].str.replace(",", ""))
nalist['Rupees'].fillna(nalist['Rup2'], inplace=True)
nalist['Dollars'].fillna(nalist['Dol2'], inplace=True)
nalist['Dollars'].fillna(nalist['Dol3'], inplace=True)
nalist['RupC'].fillna(0, inplace=True)
nalist['RupC'] = pd.to_numeric(nalist['RupC'])
nalist['RupC'] = nalist['RupC'].apply(lambda x: x*10)
nalist['Rupees'].fillna(nalist['RupC'], inplace=True)
nalist['Company'] = nalist['Company'].str.upper()
nalist['Company'] = nalist['Company'].str.replace(",", "")
nalist['Company'] = nalist['Company'].str.replace(".", "")
nalist['Foreign Collaborator'] = nalist['Foreign Collaborator'].str.upper()
nalist['Foreign Collaborator'] = nalist['Foreign Collaborator'].str.replace(",", "")
nalist['Foreign Collaborator'] = nalist['Foreign Collaborator'].str.replace(".", "")

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_18357/1210735932.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist['Company'] = nalist['Company'].str.replace(".", "")
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_18357/1210735932.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist['Foreign Collaborator'] = nalist['Foreign Collaborator'].str.replace(".", "")


In [5]:
nalist = nalist.iloc[:, 1:-4]

In [6]:
nalist['Year']=2021

In [70]:
csv_path20 = "CSVs/2020"
files = []
for file in os.listdir(csv_path20):
    if (file != ".DS_Store" and file != ".ipynb_checkpoints"):
        files.append(os.path.join(csv_path20, file))

dfs2 = []
for file in files:
    df = pd.read_csv(file)
    if(df.iloc[0][1] != "Name of Indian Company"):
        df = df.iloc[1:]
        if(df.iloc[0][1] != "Name of Indian Company"):
            df = df.iloc[1:]
    df.columns = ["#", "Company", "State", "District", "Item", "Foreign Collaborator", "Country", "Rupees", "Dollars", "Type", "NAN"]
    df['#'].astype(str)
    if(not isinstance(df.iloc[1][0], str)):
        df = df.iloc[1:]
#     print(df.iloc[1])
    #     print(df.iloc[0])
    df.iloc[1, 0] = "01"
    dfs2.append(df)
    
merged2 = pd.concat(dfs2)[1:]
merged2 = merged2.iloc[:, :-1]
merged2 = merged2.reset_index(drop=True)
merged2.drop(merged2[merged2['Company'] == 'Name of Indian Company'].index, inplace = True)
merged2.drop(merged2[merged2['Company'].isna()].index, inplace = True)
merged2 = merged2.reset_index(drop=True)
merged2

,#,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type
0,01,21CC VISHWA SARVAGANA PRIVATE LIMITED,MAHARASHTRA,Mumbai,OTHER EDUCATIONAL SERVICES N.E.C.,LOEK POSITIVE B.V.,Netherland,0.04,0.00,Received in the company
1,2,21CC VISHWA SARVAGANA PRIVATE\nLIMITED,MAHARASHTRA,Mumbai,OTHER EDUCATIONAL SERVICES N.E.C.,HR L TIEMAN,Thailand,0.04,0.00,Received in the\ncompany
2,3,21CC VISHWA SARVAGANA PRIVATE\nLIMITED,MAHARASHTRA,Mumbai,OTHER EDUCATIONAL SERVICES N.E.C.,ERJ HOLDINGS B.V,Netherland,90.75,1.20,Received in the\nCompany
3,4,3EV INDUSTRIES PRIVATE LIMITED,KARNATAKA,Bengaluru,Manufacture of motor vehicles n.e.c.,REBATT LIMITED,HongKong,10.00,0.13,Received in the Company
4,5,3i Infotech Limited,MAHARASHTRA,Thane,"Writing, modifying, testing of computer progra...",Harsh Dhar,UAE,0.17,0.00,Received in the\ncompany
...,...,...,...,...,...,...,...,...,...,...
17380,471,ZETWERK MANUFACTURING\nBUSINESSES PRIVATE LIMITED,KARNATAKA,Bengaluru,Other information technology and\ncomputer ser...,Greenoaks Capital\nOpportunities Fund II LP,U.S.A,0.52,0.01,Received in the\ncompany
17381,472,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,"ZKL Bearings CZ, Akciova Spolecnost",Czech Republic,0.05,0.00,Received in the company
17382,473,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,"ZKL Bearings CZ, Akciova Spolecnost",Czech Republic,0.05,0.00,Received in the company
17383,474,ZUPERSOFT SOLUTIONS\nPRIVATE LIMITED,TAMIL NADU,Tiruvallur,Providing software support and\nmaintenance to...,ZUPER INC,U.S.A,0.61,0.01,Received in the\nCompany


In [71]:
#Replace all the weird looking linke 

nans2 = merged2[(merged2["#"].isna())]
for i in range(len(merged2)) :
    if(isinstance(merged2.loc[i, "#"], str) and len(merged2.loc[i, "#"]) == 1):
        if (i != 0):
            if(isinstance(merged2.loc[i - 1, "#"], str) and len(merged2.loc[i - 1, "#"]) == 3):
                nans2 = nans2.append(merged2.loc[i])
inc2 = nans2[nans2["Company"].notna()].index
edited2 = merged2.drop(merged2[(merged2["#"].isna()) & (merged2['Company'].isna())].index)
edited2 = edited2.drop(edited2[edited2['Company'].isna()].index)
for num in inc2:
    edited2.loc[num-1, 'Company'] =  merged2.loc[num - 1, 'Company'] + " " + merged2.loc[num, 'Company']
    edited2.drop(labels=num, inplace=True)   
edited2 = edited2.reset_index(drop=True)
for i in edited2.index:
    edited2.loc[i, 'State'] = str(edited2.loc[i, 'State']).replace('\n',' ')
    edited2.loc[i, 'Foreign Collaborator'] = str(edited2.loc[i, 'Foreign Collaborator']).replace('\n',' ')
    edited2.loc[i, 'Company'] = str(edited2.loc[i, 'Company']).replace('\n',' ')
    edited2.loc[i, 'Type'] = str(edited2.loc[i, 'Type']).replace('\n',' ')
edited2 = edited2.iloc[1:]

In [72]:
import math

nalist2 = edited2
nalist2['Rupees'] = pd.to_numeric(nalist2['Rupees'].str.replace(",", ""))
nalist2['Dollars'] = pd.to_numeric(nalist2['Dollars'].str.replace(",", ""))
nalist2['Company'] = nalist2['Company'].str.upper()
nalist2['Company'] = nalist2['Company'].str.replace(",", "")
nalist2['Company'] = nalist2['Company'].str.replace(".", "")
nalist2['Foreign Collaborator'] = nalist2['Foreign Collaborator'].str.upper()
nalist2['Foreign Collaborator'] = nalist2['Foreign Collaborator'].str.replace(",", "")
nalist2['Foreign Collaborator'] = nalist2['Foreign Collaborator'].str.replace(".", "")
nalist2

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/1449291954.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist2['Company'] = nalist2['Company'].str.replace(".", "")
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/1449291954.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist2['Foreign Collaborator'] = nalist2['Foreign Collaborator'].str.replace(".", "")


,#,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type
1,2,21CC VISHWA SARVAGANA PRIVATE LIMITED,MAHARASHTRA,Mumbai,OTHER EDUCATIONAL SERVICES N.E.C.,HR L TIEMAN,Thailand,0.04,0.00,Received in the company
2,3,21CC VISHWA SARVAGANA PRIVATE LIMITED,MAHARASHTRA,Mumbai,OTHER EDUCATIONAL SERVICES N.E.C.,ERJ HOLDINGS BV,Netherland,90.75,1.20,Received in the Company
3,4,3EV INDUSTRIES PRIVATE LIMITED,KARNATAKA,Bengaluru,Manufacture of motor vehicles n.e.c.,REBATT LIMITED,HongKong,10.00,0.13,Received in the Company
4,5,3I INFOTECH LIMITED,MAHARASHTRA,Thane,"Writing, modifying, testing of computer progra...",HARSH DHAR,UAE,0.17,0.00,Received in the company
5,6,3TANDAI INDIA PRIVATE LIMITED,KARNATAKA,Bengaluru,Other human health activities n.e.c. (includin...,FOOD RX AND AI INC,U.S.A,0.70,0.01,Received in the company
...,...,...,...,...,...,...,...,...,...,...
16954,471,ZETWERK MANUFACTURING BUSINESSES PRIVATE LIMITED,KARNATAKA,Bengaluru,Other information technology and\ncomputer ser...,GREENOAKS CAPITAL OPPORTUNITIES FUND II LP,U.S.A,0.52,0.01,Received in the company
16955,472,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,ZKL BEARINGS CZ AKCIOVA SPOLECNOST,Czech Republic,0.05,0.00,Received in the company
16956,473,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,ZKL BEARINGS CZ AKCIOVA SPOLECNOST,Czech Republic,0.05,0.00,Received in the company
16957,474,ZUPERSOFT SOLUTIONS PRIVATE LIMITED,TAMIL NADU,Tiruvallur,Providing software support and\nmaintenance to...,ZUPER INC,U.S.A,0.61,0.01,Received in the Company


In [73]:
nalist2['Year']=2020
del nalist2['#']
nalist2

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
1,21CC VISHWA SARVAGANA PRIVATE LIMITED,MAHARASHTRA,Mumbai,OTHER EDUCATIONAL SERVICES N.E.C.,HR L TIEMAN,Thailand,0.04,0.00,Received in the company,2020
2,21CC VISHWA SARVAGANA PRIVATE LIMITED,MAHARASHTRA,Mumbai,OTHER EDUCATIONAL SERVICES N.E.C.,ERJ HOLDINGS BV,Netherland,90.75,1.20,Received in the Company,2020
3,3EV INDUSTRIES PRIVATE LIMITED,KARNATAKA,Bengaluru,Manufacture of motor vehicles n.e.c.,REBATT LIMITED,HongKong,10.00,0.13,Received in the Company,2020
4,3I INFOTECH LIMITED,MAHARASHTRA,Thane,"Writing, modifying, testing of computer progra...",HARSH DHAR,UAE,0.17,0.00,Received in the company,2020
5,3TANDAI INDIA PRIVATE LIMITED,KARNATAKA,Bengaluru,Other human health activities n.e.c. (includin...,FOOD RX AND AI INC,U.S.A,0.70,0.01,Received in the company,2020
...,...,...,...,...,...,...,...,...,...,...
16954,ZETWERK MANUFACTURING BUSINESSES PRIVATE LIMITED,KARNATAKA,Bengaluru,Other information technology and\ncomputer ser...,GREENOAKS CAPITAL OPPORTUNITIES FUND II LP,U.S.A,0.52,0.01,Received in the company,2020
16955,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,ZKL BEARINGS CZ AKCIOVA SPOLECNOST,Czech Republic,0.05,0.00,Received in the company,2020
16956,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,ZKL BEARINGS CZ AKCIOVA SPOLECNOST,Czech Republic,0.05,0.00,Received in the company,2020
16957,ZUPERSOFT SOLUTIONS PRIVATE LIMITED,TAMIL NADU,Tiruvallur,Providing software support and\nmaintenance to...,ZUPER INC,U.S.A,0.61,0.01,Received in the Company,2020


In [74]:
nalist_c = pd.concat([nalist, nalist2])
nalist_c = nalist_c.reset_index(drop=True)
nalist_c

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,ADANI INFRA (INDIA) LIMITED,GUJARAT,Gandhi Nagar,Construction and maintenance of power plants,HARMONIA TRADE AND INVESTMENT LIMITED,Mauritius,36471.99,501.29,Received in the Company,2021
1,INDUSIND BANK LIMITED,MAHARASHTRA,Pune,"Monetary intermediation of commercial\nbanks, ...",INDUSIND INTERNATIONAL HOLDINGS LIMITED,Mauritius,14858.61,204.12,Received in the Company,2021
2,PHONEPE PRIVATE LIMITED,DELHI,South Delhi,Activities auxiliary to financial service acti...,PHONEPE PRIVATE LIMITED,Singapore,11100.00,151.83,Received in the Company,2021
3,PEGATRON TECHNOLOGY INDIA PRIVATE LIMITED,TAMIL NADU,Kanchipuram,Manufacture of other communication equipments ...,PEGATRON CORPORATION,Taiwan,10448.96,143.54,Greenfield,2021
4,ZOMATO PRIVATE LIMITED,DELHI,South Delhi,Other information service activities n.e.c.,KORA HOLDINGS II C LLC,Cayman Islands,8452.47,116.12,Received in the Company,2021
...,...,...,...,...,...,...,...,...,...,...
25105,ZETWERK MANUFACTURING BUSINESSES PRIVATE LIMITED,KARNATAKA,Bengaluru,Other information technology and\ncomputer ser...,GREENOAKS CAPITAL OPPORTUNITIES FUND II LP,U.S.A,0.52,0.01,Received in the company,2020
25106,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,ZKL BEARINGS CZ AKCIOVA SPOLECNOST,Czech Republic,0.05,0.00,Received in the company,2020
25107,ZKL BEARINGS CZ INDIA PRIVATE LIMITED,MAHARASHTRA,Thane,Other retail sale of new goods in specialized ...,ZKL BEARINGS CZ AKCIOVA SPOLECNOST,Czech Republic,0.05,0.00,Received in the company,2020
25108,ZUPERSOFT SOLUTIONS PRIVATE LIMITED,TAMIL NADU,Tiruvallur,Providing software support and\nmaintenance to...,ZUPER INC,U.S.A,0.61,0.01,Received in the Company,2020


In [75]:
%store nalist_c

Stored 'nalist_c' (DataFrame)


In [76]:
csv_path19 = "CSVs/2019"
files = []
for file in os.listdir(csv_path19):
    if (file != ".DS_Store" and file != ".ipynb_checkpoints"):
        files.append(os.path.join(csv_path19, file))
        print(file)

dfs3 = []
for file in files:
    df = pd.read_csv(file)
    if(df.iloc[0][1] != "Name of Indian Company"):
        df = df.iloc[1:]
#     print(df.iloc[0])
    df.columns = ["#", "Company", "Country", "Foreign Collaborator", "Regional Office", "Item", "Rupees", "Dollars", "NAN"]
    df['#'].astype(str)
    if(not isinstance(df.iloc[1][0], str)):
        df = df.iloc[1:]
#     print(df.iloc[1])
#     print(df.iloc[0])
    df.iloc[1, 0] = "01"
    dfs3.append(df)
    

merged3 = pd.concat(dfs3)[1:-1]
merged3 = merged3.reset_index(drop=True)
merged3.drop(merged3[merged3['Company'] == 'Name of Indian Company'].index, inplace = True)
merged3.drop(merged3[merged3['Company'].isna()].index, inplace = True)
merged3 = merged3.reset_index(drop=True)
merged3

2019_april_direct.csv
2019_january_direct.csv
2019_april_rbi.csv
2019_October_RBI.csv
2019_july_shares.csv
2019_October_direct.csv
2019_October_Shares.csv
2019_july_direct.csv
2019_july_rbi.csv
2019_january_rbi.csv
2019_january_shares.csv
2019_april_shares.csv


,#,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN
0,01,EXCITEL BROADBAND PRIVATE LIMITED,Bulgaria,Intertainment Services JSCo,NEW DELHI,Activities of providing internet access by the...,160.00,2.30,NaN
1,2,Axis Bank Limited,Canada,Sanjeev Kumar Gupta,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.44,0.02,NaN
2,3,Axis Bank Limited,Canada,Sanjeev Kumar Gupta,AHMEDABAD,"Monetary intermediation of commercial banks, s...",2.31,0.03,NaN
3,4,Axis Bank Limited,Canada,Sanjeev Kumar Gupta,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.73,0.02,NaN
4,5,Axis Bank Limited,Canada,Sanjeev Kumar Gupta,AHMEDABAD,"Monetary intermediation of commercial banks, s...",3.47,0.05,NaN
...,...,...,...,...,...,...,...,...,...
22886,615,Duroflex Private Limited,Mauritius,LIGHTHOUSE INDIA FUND III LIMITED,REGION NOT\nINDICATED,Manufacture of mattresses and pillows,274.3,3.95,NaN
22887,616,INFINYPOOL ONLINE PAYMENT SOLUTIONS INDI,U.S.A,Wibmo Inc,REGION NOT INDICATED,Publishing of operating business and other app...,3.84,0.06,NaN
22888,617,BLUE JAY FINLEASE LIMITED,Mauritius,Matrix Partners India Investments II Ext,REGION NOT INDICATED,"Other financial service activities, except ins...",1.45,0.02,NaN
22889,618,Essar Steel Metal Trading Limited,Mauritius,Essar Ports & Shipping Limited,REGION NOT INDICATED,Wholesale of metals and metal ores,69.99,1.01,NaN


In [77]:
nans3 = merged3[(merged3["#"].isna())]
for i in range(len(merged3)) :
    if(isinstance(merged3.loc[i, "#"], str) and len(merged3.loc[i, "#"]) == 1):
        if (i != 0):
            if(isinstance(merged3.loc[i - 1, "#"], str) and len(merged3.loc[i - 1, "#"]) == 3):
                nans3 = nans3.append(merged3.loc[i])
inc3 = nans3[nans3["Company"].notna()].index
edited3 = merged3.drop(merged3[(merged3["#"].isna()) & (merged3['Company'].isna())].index)
edited3 = edited3.drop(edited3[edited3['Company'].isna()].index)
for num in inc3:
    for label in ['Company', 'Regional Office', 'Item']:
        if(str(merged3.loc[num, label]) != 'nan'):
            edited3.loc[num-1, label] =  merged3.loc[num - 1, label] + " " + merged3.loc[num, label]
    edited3.drop(labels=num, inplace=True)   
edited3 = edited3.reset_index(drop=True)
for i in edited3.index:
    edited3.loc[i, 'Regional Office'] = str(edited3.loc[i, 'Regional Office']).replace('\n',' ')
    edited3.loc[i, 'Foreign Collaborator'] = str(edited3.loc[i, 'Foreign Collaborator']).replace('\n',' ')
    edited3.loc[i, 'Company'] = str(edited3.loc[i, 'Company']).replace('\n',' ')

In [78]:
import math

nalist3 = edited3
nalist3['Rupees'] = pd.to_numeric(nalist3['Rupees'].str.replace(",", ""))
nalist3['Dollars'] = pd.to_numeric(nalist3['Dollars'].str.replace(",", ""))
nalist3['Company'] = nalist3['Company'].str.upper()
nalist3['Company'] = nalist3['Company'].str.replace(",", "")
nalist3['Company'] = nalist3['Company'].str.replace(".", "")
nalist3['Foreign Collaborator'] = nalist3['Foreign Collaborator'].str.upper()
nalist3['Foreign Collaborator'] = nalist3['Foreign Collaborator'].str.replace(",", "")
nalist3['Foreign Collaborator'] = nalist3['Foreign Collaborator'].str.replace(".", "")

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/2981637774.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist3['Company'] = nalist3['Company'].str.replace(".", "")
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/2981637774.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist3['Foreign Collaborator'] = nalist3['Foreign Collaborator'].str.replace(".", "")


In [79]:
nalist3['Year']=2019
del nalist3['#']
nalist3

,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN,Year
0,EXCITEL BROADBAND PRIVATE LIMITED,Bulgaria,INTERTAINMENT SERVICES JSCO,NEW DELHI,Activities of providing internet access by the...,160.00,2.30,NaN,2019
1,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.44,0.02,NaN,2019
2,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",2.31,0.03,NaN,2019
3,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.73,0.02,NaN,2019
4,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",3.47,0.05,NaN,2019
...,...,...,...,...,...,...,...,...,...
22566,DUROFLEX PRIVATE LIMITED,Mauritius,LIGHTHOUSE INDIA FUND III LIMITED,REGION NOT INDICATED,Manufacture of mattresses and pillows,274.30,3.95,NaN,2019
22567,INFINYPOOL ONLINE PAYMENT SOLUTIONS INDI,U.S.A,WIBMO INC,REGION NOT INDICATED,Publishing of operating business and other app...,3.84,0.06,NaN,2019
22568,BLUE JAY FINLEASE LIMITED,Mauritius,MATRIX PARTNERS INDIA INVESTMENTS II EXT,REGION NOT INDICATED,"Other financial service activities, except ins...",1.45,0.02,NaN,2019
22569,ESSAR STEEL METAL TRADING LIMITED,Mauritius,ESSAR PORTS & SHIPPING LIMITED,REGION NOT INDICATED,Wholesale of metals and metal ores,69.99,1.01,NaN,2019


In [80]:
nalist3.rename(columns = {'Regional Office':'District'}, inplace = True)
nalist3

,Company,Country,Foreign Collaborator,District,Item,Rupees,Dollars,NAN,Year
0,EXCITEL BROADBAND PRIVATE LIMITED,Bulgaria,INTERTAINMENT SERVICES JSCO,NEW DELHI,Activities of providing internet access by the...,160.00,2.30,NaN,2019
1,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.44,0.02,NaN,2019
2,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",2.31,0.03,NaN,2019
3,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.73,0.02,NaN,2019
4,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",3.47,0.05,NaN,2019
...,...,...,...,...,...,...,...,...,...
22566,DUROFLEX PRIVATE LIMITED,Mauritius,LIGHTHOUSE INDIA FUND III LIMITED,REGION NOT INDICATED,Manufacture of mattresses and pillows,274.30,3.95,NaN,2019
22567,INFINYPOOL ONLINE PAYMENT SOLUTIONS INDI,U.S.A,WIBMO INC,REGION NOT INDICATED,Publishing of operating business and other app...,3.84,0.06,NaN,2019
22568,BLUE JAY FINLEASE LIMITED,Mauritius,MATRIX PARTNERS INDIA INVESTMENTS II EXT,REGION NOT INDICATED,"Other financial service activities, except ins...",1.45,0.02,NaN,2019
22569,ESSAR STEEL METAL TRADING LIMITED,Mauritius,ESSAR PORTS & SHIPPING LIMITED,REGION NOT INDICATED,Wholesale of metals and metal ores,69.99,1.01,NaN,2019


In [81]:
import numpy as np
nalist3['State'] = np.nan
nalist3['Type'] = np.nan
del nalist3["NAN"]
nalist3

,Company,Country,Foreign Collaborator,District,Item,Rupees,Dollars,Year,State,Type
0,EXCITEL BROADBAND PRIVATE LIMITED,Bulgaria,INTERTAINMENT SERVICES JSCO,NEW DELHI,Activities of providing internet access by the...,160.00,2.30,2019,NaN,NaN
1,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.44,0.02,2019,NaN,NaN
2,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",2.31,0.03,2019,NaN,NaN
3,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",1.73,0.02,2019,NaN,NaN
4,AXIS BANK LIMITED,Canada,SANJEEV KUMAR GUPTA,AHMEDABAD,"Monetary intermediation of commercial banks, s...",3.47,0.05,2019,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
22566,DUROFLEX PRIVATE LIMITED,Mauritius,LIGHTHOUSE INDIA FUND III LIMITED,REGION NOT INDICATED,Manufacture of mattresses and pillows,274.30,3.95,2019,NaN,NaN
22567,INFINYPOOL ONLINE PAYMENT SOLUTIONS INDI,U.S.A,WIBMO INC,REGION NOT INDICATED,Publishing of operating business and other app...,3.84,0.06,2019,NaN,NaN
22568,BLUE JAY FINLEASE LIMITED,Mauritius,MATRIX PARTNERS INDIA INVESTMENTS II EXT,REGION NOT INDICATED,"Other financial service activities, except ins...",1.45,0.02,2019,NaN,NaN
22569,ESSAR STEEL METAL TRADING LIMITED,Mauritius,ESSAR PORTS & SHIPPING LIMITED,REGION NOT INDICATED,Wholesale of metals and metal ores,69.99,1.01,2019,NaN,NaN


In [82]:
nalist3 = nalist3[['Company', 'State', 'District', 'Item', 'Foreign Collaborator', 'Country', 'Rupees', 'Dollars', 'Type', 'Year']]


In [83]:
nalist_c = pd.concat([nalist_c, nalist3])
nalist_c = nalist_c.reset_index(drop=True)
nalist_c

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,ADANI INFRA (INDIA) LIMITED,GUJARAT,Gandhi Nagar,Construction and maintenance of power plants,HARMONIA TRADE AND INVESTMENT LIMITED,Mauritius,36471.99,501.29,Received in the Company,2021
1,INDUSIND BANK LIMITED,MAHARASHTRA,Pune,"Monetary intermediation of commercial\nbanks, ...",INDUSIND INTERNATIONAL HOLDINGS LIMITED,Mauritius,14858.61,204.12,Received in the Company,2021
2,PHONEPE PRIVATE LIMITED,DELHI,South Delhi,Activities auxiliary to financial service acti...,PHONEPE PRIVATE LIMITED,Singapore,11100.00,151.83,Received in the Company,2021
3,PEGATRON TECHNOLOGY INDIA PRIVATE LIMITED,TAMIL NADU,Kanchipuram,Manufacture of other communication equipments ...,PEGATRON CORPORATION,Taiwan,10448.96,143.54,Greenfield,2021
4,ZOMATO PRIVATE LIMITED,DELHI,South Delhi,Other information service activities n.e.c.,KORA HOLDINGS II C LLC,Cayman Islands,8452.47,116.12,Received in the Company,2021
...,...,...,...,...,...,...,...,...,...,...
47676,DUROFLEX PRIVATE LIMITED,NaN,REGION NOT INDICATED,Manufacture of mattresses and pillows,LIGHTHOUSE INDIA FUND III LIMITED,Mauritius,274.30,3.95,NaN,2019
47677,INFINYPOOL ONLINE PAYMENT SOLUTIONS INDI,NaN,REGION NOT INDICATED,Publishing of operating business and other app...,WIBMO INC,U.S.A,3.84,0.06,NaN,2019
47678,BLUE JAY FINLEASE LIMITED,NaN,REGION NOT INDICATED,"Other financial service activities, except ins...",MATRIX PARTNERS INDIA INVESTMENTS II EXT,Mauritius,1.45,0.02,NaN,2019
47679,ESSAR STEEL METAL TRADING LIMITED,NaN,REGION NOT INDICATED,Wholesale of metals and metal ores,ESSAR PORTS & SHIPPING LIMITED,Mauritius,69.99,1.01,NaN,2019


In [84]:
%store nalist_c

Stored 'nalist_c' (DataFrame)


In [85]:
csv_path18 = "CSVs/2018"
files = []
for file in os.listdir(csv_path18):
    if (file != ".DS_Store" and file != ".ipynb_checkpoints"):
        files.append(os.path.join(csv_path18, file))
        print(file)

dfs4 = []
for file in files:
    df = pd.read_csv(file)
    if(df.iloc[0][1] != "Name of Indian Company"):
        df = df.iloc[1:]
        if(df.iloc[0][1] != "Name of Indian Company"):
            df = df.iloc[1:]
            if(df.iloc[0][1] != "Name of Indian Company"):
                df = df.iloc[1:]
    df.columns = ["#", "Company", "Country", "Foreign Collaborator", "Regional Office", "Item", "Rupees", "Dollars", "NAN"]
    df['#'].astype(str)
    if(not isinstance(df.iloc[1][0], str)):
        df = df.iloc[1:]
    df.iloc[1, 0] = "01"
    dfs4.append(df)
    

merged4 = pd.concat(dfs4)[1:-1]
merged4 = merged4.reset_index(drop=True)
merged4.drop(merged4[merged4['Company'] == 'Name of Indian Company'].index, inplace = True)
merged4.drop(merged4[merged4['Company'].isna()].index, inplace = True)
merged4 = merged4.reset_index(drop=True)
merged4

2018_july_shares.csv
2018_january_shares.csv
2018_october_rbi.csv
2018_april_direct.csv
2018_october_shares.csv
2018_october_direct.csv
2018_july_rbi.csv
2018_january_rbi.csv
2018_april_rbi.csv
2018_april_shares.csv
2018_january_direct.csv
2018_july_direct.csv


,#,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN
0,01,Aptus Value Housing Finance India Ltd,Mauritius,"Westbridge Crossover Fund, Llc, Mauritiu",REGION NOT INDICATED,"Other financial service activities, except ins...",7.85,0.12,NaN
1,2,Aptus Value Housing Finance India Ltd,Mauritius,"Westbridge Crossover Fund, Llc, Mauritiu",REGION\nNOT INDICATED,"Other financial service activities,\nexcept in...",15.70,0.23,NaN
2,3,Bentoli Agrinutrition India Pvt Ltd,U.S.A,"William Robinson, Usa",REGION NOT INDICATED,Manufacture of poultry feed,0.90,0.01,NaN
3,4,Bentoli Agrinutrition India Pvt Ltd,U.S.A,Edward S Robinson. Usa,REGION NOT INDICATED,Manufacture of poultry feed,0.90,0.01,NaN
4,5,Iewc Wire & Cable India Pvt Ltd,U.S.A,"Iewc Holding Corp ,Usa",REGION NOT INDICATED,Activities of the cable operators,1.00,0.01,NaN
...,...,...,...,...,...,...,...,...,...
17589,40,AIRIS PHARMA PVT LTD,U.S.A,Kishore Kumar Ganji,HYDERABAD,Research and experimental development on natur...,0.94,0.01,NaN
17590,41,AIRIS PHARMA PVT LTD,U.S.A,Kishore Kumar Ganji,HYDERABAD,Research and experimental development on natur...,4.00,0.06,NaN
17591,42,AIRIS PHARMA PVT LTD,U.S.A,Mohan R Patalolla,HYDERABAD,Research and experimental\ndevelopment on natu...,13.39,0.20,NaN
17592,43,AIRIS PHARMA PVT LTD,U.S.A,Kishore Kumar Gani,HYDERABAD,Research and experimental development on natur...,11.00,0.17,NaN


In [86]:
nans4 = merged4[(merged4["#"].isna())]
nans4
for i in range(len(merged4)) :
    if(isinstance(merged4.loc[i, "#"], str) and len(merged4.loc[i, "#"]) == 1):
        if (i != 0):
            if(isinstance(merged4.loc[i - 1, "#"], str) and len(merged4.loc[i - 1, "#"]) == 3):
                nans4 = nans4.append(merged4.loc[i])
inc4 = nans4[nans4["Company"].notna()].index
edited4 = merged4.drop(merged4[(merged4["#"].isna()) & (merged4['Company'].isna())].index)
edited4 = edited4.drop(edited4[edited4['Company'].isna()].index)
edited4
for num in inc4:
    for label in ['Company', 'Foreign Collaborator', 'Regional Office', 'Item']:
        if(str(merged4.loc[num, label]) != 'nan'):
            edited4.loc[num-1, label] =  merged4.loc[num - 1, label] + " " + merged4.loc[num, label]
    edited4.drop(labels=num, inplace=True)   
edited4 = edited4.reset_index(drop=True)
for i in edited4.index:
    edited4.loc[i, 'Regional Office'] = str(edited4.loc[i, 'Regional Office']).replace('\n',' ')
    edited4.loc[i, 'Foreign Collaborator'] = str(edited4.loc[i, 'Foreign Collaborator']).replace('\n',' ')
    edited4.loc[i, 'Company'] = str(edited4.loc[i, 'Company']).replace('\n',' ')
    edited4.loc[i, 'Rupees'] = str(edited4.loc[i, 'Rupees']).replace('\n','')
    edited4.loc[i, 'Dollars'] = str(edited4.loc[i, 'Dollars']).replace('\n','')

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/4251658019.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans4 = nans4.append(merged4.loc[i])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/4251658019.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans4 = nans4.append(merged4.loc[i])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/4251658019.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans4 = nans4.append(merged4.loc[i])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/4251658019.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans4 = nans4.append(

In [87]:
edited4.iloc[9970]

#                                                                     103
Company                          CERES BIOSYSTEMS INDIA PVT LTD(Erstwhile
Country                                                            Canada
Foreign Collaborator                                 Ceres Biosystems Ltd
Regional Office                                                 AHMEDABAD
Item                    Manufacture of urea and other organic fertilizers
Rupees                                                               5.07
Dollars                                                              0.08
NAN                                                                   NaN
Name: 9970, dtype: object

In [88]:
import math

nalist4 = edited4
nalist4['Rupees'] = pd.to_numeric(nalist4['Rupees'].str.replace(",", ""))
nalist4['Dollars'] = pd.to_numeric(nalist4['Dollars'].str.replace(",", ""))

In [89]:
nalist4['Company'] = nalist4['Company'].str.upper()
nalist4['Company'] = nalist4['Company'].str.replace(",", "")
nalist4['Company'] = nalist4['Company'].str.replace(".", "")
nalist4['Foreign Collaborator'] = nalist4['Foreign Collaborator'].str.upper()
nalist4['Foreign Collaborator'] = nalist4['Foreign Collaborator'].str.replace(",", "")
nalist4['Foreign Collaborator'] = nalist4['Foreign Collaborator'].str.replace(".", "")
nalist4['Year']=2018
del nalist4['#']
nalist4

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/4057128038.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist4['Company'] = nalist4['Company'].str.replace(".", "")
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/4057128038.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist4['Foreign Collaborator'] = nalist4['Foreign Collaborator'].str.replace(".", "")


,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN,Year
0,APTUS VALUE HOUSING FINANCE INDIA LTD,Mauritius,WESTBRIDGE CROSSOVER FUND LLC MAURITIU,REGION NOT INDICATED,"Other financial service activities, except ins...",7.85,0.12,NaN,2018
1,APTUS VALUE HOUSING FINANCE INDIA LTD,Mauritius,WESTBRIDGE CROSSOVER FUND LLC MAURITIU,REGION NOT INDICATED,"Other financial service activities,\nexcept in...",15.70,0.23,NaN,2018
2,BENTOLI AGRINUTRITION INDIA PVT LTD,U.S.A,WILLIAM ROBINSON USA,REGION NOT INDICATED,Manufacture of poultry feed,0.90,0.01,NaN,2018
3,BENTOLI AGRINUTRITION INDIA PVT LTD,U.S.A,EDWARD S ROBINSON USA,REGION NOT INDICATED,Manufacture of poultry feed,0.90,0.01,NaN,2018
4,IEWC WIRE & CABLE INDIA PVT LTD,U.S.A,IEWC HOLDING CORP USA,REGION NOT INDICATED,Activities of the cable operators,1.00,0.01,NaN,2018
...,...,...,...,...,...,...,...,...,...
17367,AIRIS PHARMA PVT LTD,U.S.A,KISHORE KUMAR GANJI,HYDERABAD,Research and experimental development on natur...,0.94,0.01,NaN,2018
17368,AIRIS PHARMA PVT LTD,U.S.A,KISHORE KUMAR GANJI,HYDERABAD,Research and experimental development on natur...,4.00,0.06,NaN,2018
17369,AIRIS PHARMA PVT LTD,U.S.A,MOHAN R PATALOLLA,HYDERABAD,Research and experimental\ndevelopment on natu...,13.39,0.20,NaN,2018
17370,AIRIS PHARMA PVT LTD,U.S.A,KISHORE KUMAR GANI,HYDERABAD,Research and experimental development on natur...,11.00,0.17,NaN,2018


In [90]:
import numpy as np

nalist4.rename(columns = {'Regional Office':'District'}, inplace = True)
nalist4['State'] = np.nan
nalist4['Type'] = np.nan
del nalist4["NAN"]
nalist4

,Company,Country,Foreign Collaborator,District,Item,Rupees,Dollars,Year,State,Type
0,APTUS VALUE HOUSING FINANCE INDIA LTD,Mauritius,WESTBRIDGE CROSSOVER FUND LLC MAURITIU,REGION NOT INDICATED,"Other financial service activities, except ins...",7.85,0.12,2018,NaN,NaN
1,APTUS VALUE HOUSING FINANCE INDIA LTD,Mauritius,WESTBRIDGE CROSSOVER FUND LLC MAURITIU,REGION NOT INDICATED,"Other financial service activities,\nexcept in...",15.70,0.23,2018,NaN,NaN
2,BENTOLI AGRINUTRITION INDIA PVT LTD,U.S.A,WILLIAM ROBINSON USA,REGION NOT INDICATED,Manufacture of poultry feed,0.90,0.01,2018,NaN,NaN
3,BENTOLI AGRINUTRITION INDIA PVT LTD,U.S.A,EDWARD S ROBINSON USA,REGION NOT INDICATED,Manufacture of poultry feed,0.90,0.01,2018,NaN,NaN
4,IEWC WIRE & CABLE INDIA PVT LTD,U.S.A,IEWC HOLDING CORP USA,REGION NOT INDICATED,Activities of the cable operators,1.00,0.01,2018,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
17367,AIRIS PHARMA PVT LTD,U.S.A,KISHORE KUMAR GANJI,HYDERABAD,Research and experimental development on natur...,0.94,0.01,2018,NaN,NaN
17368,AIRIS PHARMA PVT LTD,U.S.A,KISHORE KUMAR GANJI,HYDERABAD,Research and experimental development on natur...,4.00,0.06,2018,NaN,NaN
17369,AIRIS PHARMA PVT LTD,U.S.A,MOHAN R PATALOLLA,HYDERABAD,Research and experimental\ndevelopment on natu...,13.39,0.20,2018,NaN,NaN
17370,AIRIS PHARMA PVT LTD,U.S.A,KISHORE KUMAR GANI,HYDERABAD,Research and experimental development on natur...,11.00,0.17,2018,NaN,NaN


In [91]:
nalist4 = nalist4[['Company', 'State', 'District', 'Item', 'Foreign Collaborator', 'Country', 'Rupees', 'Dollars', 'Type', 'Year']]
nalist4

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,APTUS VALUE HOUSING FINANCE INDIA LTD,NaN,REGION NOT INDICATED,"Other financial service activities, except ins...",WESTBRIDGE CROSSOVER FUND LLC MAURITIU,Mauritius,7.85,0.12,NaN,2018
1,APTUS VALUE HOUSING FINANCE INDIA LTD,NaN,REGION NOT INDICATED,"Other financial service activities,\nexcept in...",WESTBRIDGE CROSSOVER FUND LLC MAURITIU,Mauritius,15.70,0.23,NaN,2018
2,BENTOLI AGRINUTRITION INDIA PVT LTD,NaN,REGION NOT INDICATED,Manufacture of poultry feed,WILLIAM ROBINSON USA,U.S.A,0.90,0.01,NaN,2018
3,BENTOLI AGRINUTRITION INDIA PVT LTD,NaN,REGION NOT INDICATED,Manufacture of poultry feed,EDWARD S ROBINSON USA,U.S.A,0.90,0.01,NaN,2018
4,IEWC WIRE & CABLE INDIA PVT LTD,NaN,REGION NOT INDICATED,Activities of the cable operators,IEWC HOLDING CORP USA,U.S.A,1.00,0.01,NaN,2018
...,...,...,...,...,...,...,...,...,...,...
17367,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental development on natur...,KISHORE KUMAR GANJI,U.S.A,0.94,0.01,NaN,2018
17368,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental development on natur...,KISHORE KUMAR GANJI,U.S.A,4.00,0.06,NaN,2018
17369,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental\ndevelopment on natu...,MOHAN R PATALOLLA,U.S.A,13.39,0.20,NaN,2018
17370,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental development on natur...,KISHORE KUMAR GANI,U.S.A,11.00,0.17,NaN,2018


In [92]:
nalist_c = pd.concat([nalist_c, nalist4])
nalist_c = nalist_c.reset_index(drop=True)
nalist_c

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,ADANI INFRA (INDIA) LIMITED,GUJARAT,Gandhi Nagar,Construction and maintenance of power plants,HARMONIA TRADE AND INVESTMENT LIMITED,Mauritius,36471.99,501.29,Received in the Company,2021
1,INDUSIND BANK LIMITED,MAHARASHTRA,Pune,"Monetary intermediation of commercial\nbanks, ...",INDUSIND INTERNATIONAL HOLDINGS LIMITED,Mauritius,14858.61,204.12,Received in the Company,2021
2,PHONEPE PRIVATE LIMITED,DELHI,South Delhi,Activities auxiliary to financial service acti...,PHONEPE PRIVATE LIMITED,Singapore,11100.00,151.83,Received in the Company,2021
3,PEGATRON TECHNOLOGY INDIA PRIVATE LIMITED,TAMIL NADU,Kanchipuram,Manufacture of other communication equipments ...,PEGATRON CORPORATION,Taiwan,10448.96,143.54,Greenfield,2021
4,ZOMATO PRIVATE LIMITED,DELHI,South Delhi,Other information service activities n.e.c.,KORA HOLDINGS II C LLC,Cayman Islands,8452.47,116.12,Received in the Company,2021
...,...,...,...,...,...,...,...,...,...,...
65048,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental development on natur...,KISHORE KUMAR GANJI,U.S.A,0.94,0.01,NaN,2018
65049,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental development on natur...,KISHORE KUMAR GANJI,U.S.A,4.00,0.06,NaN,2018
65050,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental\ndevelopment on natu...,MOHAN R PATALOLLA,U.S.A,13.39,0.20,NaN,2018
65051,AIRIS PHARMA PVT LTD,NaN,HYDERABAD,Research and experimental development on natur...,KISHORE KUMAR GANI,U.S.A,11.00,0.17,NaN,2018


In [93]:
%store nalist_c

Stored 'nalist_c' (DataFrame)


In [94]:
csv_path17 = "CSVs/2017"
files = []
for file in os.listdir(csv_path17):
    if (file != ".DS_Store" and file != ".ipynb_checkpoints"):
        files.append(os.path.join(csv_path17, file))
        print(file)

dfs5 = []
for file in files:
    df = pd.read_csv(file)
    if(df.iloc[0][1] != "Name of Indian Company"):
        df = df.iloc[1:]
        if(df.iloc[0][1] != "Name of Indian Company"):
            df = df.iloc[1:]
            if(df.iloc[0][1] != "Name of Indian Company"):
                df = df.iloc[1:]
    df.columns = ["#", "Company", "Country", "Foreign Collaborator", "Regional Office", "Item", "Rupees", "Dollars", "NAN"]
    df['#'].astype(str)
    if(not isinstance(df.iloc[1][0], str)):
        df = df.iloc[1:]
    df.iloc[1, 0] = "01"
    dfs5.append(df)
    

merged5 = pd.concat(dfs5)[1:-1]
merged5 = merged5.reset_index(drop=True)
merged5.drop(merged5[merged5['Company'] == 'Name of Indian Company'].index, inplace = True)
merged5.drop(merged5[merged5['Company'].isna()].index, inplace = True)
merged5 = merged5.reset_index(drop=True)
merged5

2017_january_direct.csv
2017_july_shares.csv
2017_october_RBI.csv
2017_April_RBI.csv
2017_july_rbi.csv
2017_october_direct.csv
2017_April_direct.csv
2017_April_shares.csv
2017_january_rbi.csv
2017_october_shares.csv
2017_January_shares.csv
2017_july_direct.csv


,#,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN
0,01,M/S DELSEY INDIA PVT. LTD,France,DELSEY S.A.,NEW DELHI,Other non-specialised wholesale trade n.e.c.,10.20,0.15,NaN
1,2,DECATHLON SPORTS INDIA PVT LTD,France,DECATHLON SA,BANGALORE,Retail sale of sporting equipment in specializ...,"1,088.23",16.09,NaN
2,3,ROBERT BOSCH\nAUTOMOTIVE STEERING PVT LTD,Germany,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Manufacture of diverse parts and accessories\n...,88.80,1.31,NaN
3,4,ROBERT BOSCH AUTOMOTIVE STEERING\nPVT LTD,Germany,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Manufacture of diverse parts and accessories f...,111.00,1.64,NaN
4,5,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,Germany,ZF LENKSYSTEME GMBH,MUMBAI,Manufacture of diverse parts and accessories f...,429.20,6.35,NaN
...,...,...,...,...,...,...,...,...,...
16642,80,IAPMO PLUMBING CODES & STANDARDS INDIA P,U.S.A,INTERNATIONAL ASSOCIATION OF PLMBING AND,BANGALORE,Technical testing and analysis,2.00,0.03,NaN
16643,81,NIKE INDIA (P) LTD.,U.S.A,Nike Inc,BANGALORE,Retail sale of sporting equipment in specializ...,10.00,0.16,NaN
16644,82,NIKE INDIA (P) LTD.,U.S.A,Nike Holdings Inc,BANGALORE,Retail sale of sporting equipment in specializ...,0.00,0.00,NaN
16645,83,READERS DIGEST BOOK & HOME ENTERTAINMENT,U.S.A,TRUSTED MEDIA BRANDS INC,NEW DELHI,"Printing of magazines and other periodicals, b...",9.94,0.15,NaN


In [95]:
nans5 = merged5[(merged5["#"].isna())]
nans5
for i in range(len(merged5)) :
    if(isinstance(merged5.loc[i, "#"], str) and len(merged5.loc[i, "#"]) == 1):
        if (i != 0):
            if(isinstance(merged5.loc[i - 1, "#"], str) and len(merged5.loc[i - 1, "#"]) == 3):
                nans5 = nans5.append(merged5.loc[i])
inc5 = nans5[nans5["Company"].notna()].index
edited5 = merged5.drop(merged5[(merged5["#"].isna()) & (merged5['Company'].isna())].index)
edited5 = edited5.drop(edited5[edited5['Company'].isna()].index)
for num in inc5:
    for label in ['Company', 'Foreign Collaborator', 'Regional Office', 'Item']:
        if(str(merged5.loc[num, label]) != 'nan'):
            edited5.loc[num-1, label] =  merged5.loc[num - 1, label] + " " + merged5.loc[num, label]
    edited5.drop(labels=num, inplace=True)   
edited5 = edited5.reset_index(drop=True)
for i in edited5.index:
    edited5.loc[i, 'Regional Office'] = str(edited5.loc[i, 'Regional Office']).replace('\n',' ')
    edited5.loc[i, 'Foreign Collaborator'] = str(edited5.loc[i, 'Foreign Collaborator']).replace('\n',' ')
    edited5.loc[i, 'Company'] = str(edited5.loc[i, 'Company']).replace('\n',' ')
    edited5.loc[i, 'Rupees'] = str(edited5.loc[i, 'Rupees']).replace('\n','')
    edited5.loc[i, 'Dollars'] = str(edited5.loc[i, 'Dollars']).replace('\n','')

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/1801505678.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans5 = nans5.append(merged5.loc[i])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/1801505678.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans5 = nans5.append(merged5.loc[i])


In [96]:
edited5.iloc[12058]

#                                                           239
Company                 Sanjay Maintenance Services Private Lim
Country                                               Mauritius
Foreign Collaborator           Launcelot Investments, Mauritius
Regional Office                            REGION NOT INDICATED
Item                              General cleaning of buildings
Rupees                                                     3.93
Dollars                                                    0.06
NAN                                                         NaN
Name: 12058, dtype: object

In [97]:
import math

nalist5 = edited5
nalist5['Rupees'] = pd.to_numeric(nalist5['Rupees'].str.replace(",", ""))
nalist5['Dollars'] = pd.to_numeric(nalist5['Dollars'].str.replace(",", ""))

In [98]:
nalist5['Company'] = nalist5['Company'].str.upper()
nalist5['Company'] = nalist5['Company'].str.replace(",", "")
nalist5['Company'] = nalist5['Company'].str.replace(".", "")
nalist5['Foreign Collaborator'] = nalist5['Foreign Collaborator'].str.upper()
nalist5['Foreign Collaborator'] = nalist5['Foreign Collaborator'].str.replace(",", "")
nalist5['Foreign Collaborator'] = nalist5['Foreign Collaborator'].str.replace(".", "")
nalist5['Year']=2017
del nalist5['#']
nalist5

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/3423450870.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist5['Company'] = nalist5['Company'].str.replace(".", "")
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/3423450870.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist5['Foreign Collaborator'] = nalist5['Foreign Collaborator'].str.replace(".", "")


,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN,Year
0,M/S DELSEY INDIA PVT LTD,France,DELSEY SA,NEW DELHI,Other non-specialised wholesale trade n.e.c.,10.20,0.15,NaN,2017
1,DECATHLON SPORTS INDIA PVT LTD,France,DECATHLON SA,BANGALORE,Retail sale of sporting equipment in specializ...,1088.23,16.09,NaN,2017
2,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,Germany,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Manufacture of diverse parts and accessories\n...,88.80,1.31,NaN,2017
3,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,Germany,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Manufacture of diverse parts and accessories f...,111.00,1.64,NaN,2017
4,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,Germany,ZF LENKSYSTEME GMBH,MUMBAI,Manufacture of diverse parts and accessories f...,429.20,6.35,NaN,2017
...,...,...,...,...,...,...,...,...,...
16447,IAPMO PLUMBING CODES & STANDARDS INDIA P,U.S.A,INTERNATIONAL ASSOCIATION OF PLMBING AND,BANGALORE,Technical testing and analysis,2.00,0.03,NaN,2017
16448,NIKE INDIA (P) LTD,U.S.A,NIKE INC,BANGALORE,Retail sale of sporting equipment in specializ...,10.00,0.16,NaN,2017
16449,NIKE INDIA (P) LTD,U.S.A,NIKE HOLDINGS INC,BANGALORE,Retail sale of sporting equipment in specializ...,0.00,0.00,NaN,2017
16450,READERS DIGEST BOOK & HOME ENTERTAINMENT,U.S.A,TRUSTED MEDIA BRANDS INC,NEW DELHI,"Printing of magazines and other periodicals, b...",9.94,0.15,NaN,2017


In [99]:
import numpy as np

nalist5.rename(columns = {'Regional Office':'District'}, inplace = True)
nalist5['State'] = np.nan
nalist5['Type'] = np.nan
del nalist5["NAN"]
nalist5

,Company,Country,Foreign Collaborator,District,Item,Rupees,Dollars,Year,State,Type
0,M/S DELSEY INDIA PVT LTD,France,DELSEY SA,NEW DELHI,Other non-specialised wholesale trade n.e.c.,10.20,0.15,2017,NaN,NaN
1,DECATHLON SPORTS INDIA PVT LTD,France,DECATHLON SA,BANGALORE,Retail sale of sporting equipment in specializ...,1088.23,16.09,2017,NaN,NaN
2,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,Germany,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Manufacture of diverse parts and accessories\n...,88.80,1.31,2017,NaN,NaN
3,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,Germany,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Manufacture of diverse parts and accessories f...,111.00,1.64,2017,NaN,NaN
4,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,Germany,ZF LENKSYSTEME GMBH,MUMBAI,Manufacture of diverse parts and accessories f...,429.20,6.35,2017,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
16447,IAPMO PLUMBING CODES & STANDARDS INDIA P,U.S.A,INTERNATIONAL ASSOCIATION OF PLMBING AND,BANGALORE,Technical testing and analysis,2.00,0.03,2017,NaN,NaN
16448,NIKE INDIA (P) LTD,U.S.A,NIKE INC,BANGALORE,Retail sale of sporting equipment in specializ...,10.00,0.16,2017,NaN,NaN
16449,NIKE INDIA (P) LTD,U.S.A,NIKE HOLDINGS INC,BANGALORE,Retail sale of sporting equipment in specializ...,0.00,0.00,2017,NaN,NaN
16450,READERS DIGEST BOOK & HOME ENTERTAINMENT,U.S.A,TRUSTED MEDIA BRANDS INC,NEW DELHI,"Printing of magazines and other periodicals, b...",9.94,0.15,2017,NaN,NaN


In [100]:
nalist5 = nalist5[['Company', 'State', 'District', 'Item', 'Foreign Collaborator', 'Country', 'Rupees', 'Dollars', 'Type', 'Year']]
nalist5

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,M/S DELSEY INDIA PVT LTD,NaN,NEW DELHI,Other non-specialised wholesale trade n.e.c.,DELSEY SA,France,10.20,0.15,NaN,2017
1,DECATHLON SPORTS INDIA PVT LTD,NaN,BANGALORE,Retail sale of sporting equipment in specializ...,DECATHLON SA,France,1088.23,16.09,NaN,2017
2,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,NaN,MUMBAI,Manufacture of diverse parts and accessories\n...,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,Germany,88.80,1.31,NaN,2017
3,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,NaN,MUMBAI,Manufacture of diverse parts and accessories f...,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,Germany,111.00,1.64,NaN,2017
4,ROBERT BOSCH AUTOMOTIVE STEERING PVT LTD,NaN,MUMBAI,Manufacture of diverse parts and accessories f...,ZF LENKSYSTEME GMBH,Germany,429.20,6.35,NaN,2017
...,...,...,...,...,...,...,...,...,...,...
16447,IAPMO PLUMBING CODES & STANDARDS INDIA P,NaN,BANGALORE,Technical testing and analysis,INTERNATIONAL ASSOCIATION OF PLMBING AND,U.S.A,2.00,0.03,NaN,2017
16448,NIKE INDIA (P) LTD,NaN,BANGALORE,Retail sale of sporting equipment in specializ...,NIKE INC,U.S.A,10.00,0.16,NaN,2017
16449,NIKE INDIA (P) LTD,NaN,BANGALORE,Retail sale of sporting equipment in specializ...,NIKE HOLDINGS INC,U.S.A,0.00,0.00,NaN,2017
16450,READERS DIGEST BOOK & HOME ENTERTAINMENT,NaN,NEW DELHI,"Printing of magazines and other periodicals, b...",TRUSTED MEDIA BRANDS INC,U.S.A,9.94,0.15,NaN,2017


In [101]:
nalist_c = pd.concat([nalist_c, nalist5])
nalist_c = nalist_c.reset_index(drop=True)
nalist_c

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,ADANI INFRA (INDIA) LIMITED,GUJARAT,Gandhi Nagar,Construction and maintenance of power plants,HARMONIA TRADE AND INVESTMENT LIMITED,Mauritius,36471.99,501.29,Received in the Company,2021
1,INDUSIND BANK LIMITED,MAHARASHTRA,Pune,"Monetary intermediation of commercial\nbanks, ...",INDUSIND INTERNATIONAL HOLDINGS LIMITED,Mauritius,14858.61,204.12,Received in the Company,2021
2,PHONEPE PRIVATE LIMITED,DELHI,South Delhi,Activities auxiliary to financial service acti...,PHONEPE PRIVATE LIMITED,Singapore,11100.00,151.83,Received in the Company,2021
3,PEGATRON TECHNOLOGY INDIA PRIVATE LIMITED,TAMIL NADU,Kanchipuram,Manufacture of other communication equipments ...,PEGATRON CORPORATION,Taiwan,10448.96,143.54,Greenfield,2021
4,ZOMATO PRIVATE LIMITED,DELHI,South Delhi,Other information service activities n.e.c.,KORA HOLDINGS II C LLC,Cayman Islands,8452.47,116.12,Received in the Company,2021
...,...,...,...,...,...,...,...,...,...,...
81500,IAPMO PLUMBING CODES & STANDARDS INDIA P,NaN,BANGALORE,Technical testing and analysis,INTERNATIONAL ASSOCIATION OF PLMBING AND,U.S.A,2.00,0.03,NaN,2017
81501,NIKE INDIA (P) LTD,NaN,BANGALORE,Retail sale of sporting equipment in specializ...,NIKE INC,U.S.A,10.00,0.16,NaN,2017
81502,NIKE INDIA (P) LTD,NaN,BANGALORE,Retail sale of sporting equipment in specializ...,NIKE HOLDINGS INC,U.S.A,0.00,0.00,NaN,2017
81503,READERS DIGEST BOOK & HOME ENTERTAINMENT,NaN,NEW DELHI,"Printing of magazines and other periodicals, b...",TRUSTED MEDIA BRANDS INC,U.S.A,9.94,0.15,NaN,2017


In [102]:
%store nalist_c

Stored 'nalist_c' (DataFrame)


In [103]:
csv_path16 = "CSVs/2016"
files = []
for file in os.listdir(csv_path16):
    if (file != ".DS_Store" and file != ".ipynb_checkpoints"):
        files.append(os.path.join(csv_path16, file))
        print(file)

dfs6 = []
for file in files:
    df = pd.read_csv(file)
    if(df.iloc[0][1] != "Name of Indian Company"):
        df = df.iloc[1:]
        if(df.iloc[0][1] != "Name of Indian Company"):
            df = df.iloc[1:]
            if(df.iloc[0][1] != "Name of Indian Company"):
                df = df.iloc[1:]
    df.columns = ["#", "Company", "Country", "Foreign Collaborator", "Regional Office", "Item", "Rupees", "Dollars", "NAN"]
    df['#'].astype(str)
    if(not isinstance(df.iloc[1][0], str)):
        df = df.iloc[1:]
    df.iloc[1, 0] = "01"
    dfs6.append(df)
    

merged6 = pd.concat(dfs6)[1:-1]
merged6 = merged6.reset_index(drop=True)
merged6.drop(merged6[merged6['Company'] == 'Name of Indian Company'].index, inplace = True)
merged6.drop(merged6[merged6['Company'].isna()].index, inplace = True)
merged6 = merged6.reset_index(drop=True)
merged6

2016_July_RBI.csv
2016_January_shares.csv
2016_October_shares.csv
2016_October_RBI.csv
2016_July_Shares.csv
2016_january_FIPB.csv
2016_october_FIPB.csv
2016_April_FIPB.csv
2016_April_Shares.csv
2016_January_RBI.csv
2016_April_RBI.csv
2016_July_FIPB.csv


,#,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN
0,01,PHARMACHOL CHEMICALS PRIVATE LIMITED,Australia,MARAMEL PTY LTD,NEW DELHI,Manufacture of other articles n.e.c.,14.84,0.22,NaN
1,2,VETEA INDIA PRIVATE LIMITED,Australia,"VOCATIONAL EDUCATION, TRAINING AND EMPLO",NEW DELHI,Sports and recreation education,0.39,0.01,NaN
2,3,MKA ENGINEERS AND EXPORTERS PVT LTD,Australia,khodaram Behram Pollard,MUMBAI,Other business support service activities n.e.c.,15.00,0.22,NaN
3,4,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,Australia,Chromaflo Technologies Australia Pty Ltd,MUMBAI,"Manufacture of paints and varnishes, enamels o...",2.88,0.04,NaN
4,5,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,Australia,Srinivasa Rao,MUMBAI,"Manufacture of paints and varnishes, enamels o...",0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...
15477,70,VAL PRODUCTS INDIA PVT COMPANY,U.S.A,VAL PRODUCTS INC,MUMBAI,Manufacture of other dairy products n.e.c.,11.93,0.18,NaN
15478,71,FOX CHANNELS (INDIA) PVT. LTD.,U.S.A,FOX INTERNATIONAL CHANNELS (US) INC,MUMBAI,Radio broadcasting,15.79,0.24,NaN
15479,72,FOX CHANNELS (INDIA) PVT. LTD.,U.S.A,FOX INTL CHANNELS (US) INC,MUMBAI,Radio broadcasting,39.10,0.59,NaN
15480,73,FOX CHANNELS (INDIA) PVT. LTD.,U.S.A,FOX INTL CHANNELS (US) INC,MUMBAI,Radio broadcasting,42.09,0.63,NaN


In [104]:
nans6 = merged6[(merged6["#"].isna())]
nans6
for i in range(len(merged6)) :
    if(isinstance(merged6.loc[i, "#"], str) and len(merged6.loc[i, "#"]) == 1):
        if (i != 0):
            if(isinstance(merged6.loc[i - 1, "#"], str) and len(merged6.loc[i - 1, "#"]) == 3):
                nans6 = nans6.append(merged6.loc[i])
inc6 = nans6[nans6["Company"].notna()].index
edited6 = merged6.drop(merged6[(merged6["#"].isna()) & (merged6['Company'].isna())].index)
edited6 = edited6.drop(edited6[edited6['Company'].isna()].index)
for num in inc6:
    for label in ['Company', 'Foreign Collaborator', 'Regional Office', 'Item']:
        if(str(merged6.loc[num, label]) != 'nan'):
            edited6.loc[num-1, label] =  merged6.loc[num - 1, label] + " " + merged6.loc[num, label]
    edited6.drop(labels=num, inplace=True)   
edited6 = edited6.reset_index(drop=True)
for i in edited6.index:
    edited6.loc[i, 'Regional Office'] = str(edited6.loc[i, 'Regional Office']).replace('\n',' ')
    edited6.loc[i, 'Foreign Collaborator'] = str(edited5.loc[i, 'Foreign Collaborator']).replace('\n',' ')
    edited6.loc[i, 'Company'] = str(edited6.loc[i, 'Company']).replace('\n',' ')
    edited6.loc[i, 'Rupees'] = str(edited6.loc[i, 'Rupees']).replace('\n','')
    edited6.loc[i, 'Dollars'] = str(edited6.loc[i, 'Dollars']).replace('\n','')

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/2817111998.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans6 = nans6.append(merged6.loc[i])
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/2817111998.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nans6 = nans6.append(merged6.loc[i])


In [105]:
import math

nalist6 = edited6
nalist6['Rupees'] = pd.to_numeric(nalist6['Rupees'].str.replace(",", ""))
nalist6['Dollars'] = pd.to_numeric(nalist6['Dollars'].str.replace(",", ""))

In [106]:
nalist6['Company'] = nalist6['Company'].str.upper()
nalist6['Company'] = nalist6['Company'].str.replace(",", "")
nalist6['Company'] = nalist6['Company'].str.replace(".", "")
nalist6['Foreign Collaborator'] = nalist6['Foreign Collaborator'].str.upper()
nalist6['Foreign Collaborator'] = nalist6['Foreign Collaborator'].str.replace(",", "")
nalist6['Foreign Collaborator'] = nalist6['Foreign Collaborator'].str.replace(".", "")
nalist6['Year']=2016
del nalist6['#']
nalist6

/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/308652465.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist6['Company'] = nalist6['Company'].str.replace(".", "")
/var/folders/0n/lrk8qbzx5957h5wbrqq_707m0000gn/T/ipykernel_27669/308652465.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nalist6['Foreign Collaborator'] = nalist6['Foreign Collaborator'].str.replace(".", "")


,Company,Country,Foreign Collaborator,Regional Office,Item,Rupees,Dollars,NAN,Year
0,PHARMACHOL CHEMICALS PRIVATE LIMITED,Australia,DELSEY SA,NEW DELHI,Manufacture of other articles n.e.c.,14.84,0.22,NaN,2016
1,VETEA INDIA PRIVATE LIMITED,Australia,DECATHLON SA,NEW DELHI,Sports and recreation education,0.39,0.01,NaN,2016
2,MKA ENGINEERS AND EXPORTERS PVT LTD,Australia,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Other business support service activities n.e.c.,15.00,0.22,NaN,2016
3,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,Australia,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,"Manufacture of paints and varnishes, enamels o...",2.88,0.04,NaN,2016
4,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,Australia,ZF LENKSYSTEME GMBH,MUMBAI,"Manufacture of paints and varnishes, enamels o...",0.00,0.00,NaN,2016
...,...,...,...,...,...,...,...,...,...
15299,VAL PRODUCTS INDIA PVT COMPANY,U.S.A,CAMERON JAMES ALBORZIAN,MUMBAI,Manufacture of other dairy products n.e.c.,11.93,0.18,NaN,2016
15300,FOX CHANNELS (INDIA) PVT LTD,U.S.A,JAIYMSE MAYUMI HAFT,MUMBAI,Radio broadcasting,15.79,0.24,NaN,2016
15301,FOX CHANNELS (INDIA) PVT LTD,U.S.A,DHIRAJ SHAH,MUMBAI,Radio broadcasting,39.10,0.59,NaN,2016
15302,FOX CHANNELS (INDIA) PVT LTD,U.S.A,NAMRATA SHAH,MUMBAI,Radio broadcasting,42.09,0.63,NaN,2016


In [107]:
import numpy as np

nalist6.rename(columns = {'Regional Office':'District'}, inplace = True)
nalist6['State'] = np.nan
nalist6['Type'] = np.nan
del nalist6["NAN"]
nalist6

,Company,Country,Foreign Collaborator,District,Item,Rupees,Dollars,Year,State,Type
0,PHARMACHOL CHEMICALS PRIVATE LIMITED,Australia,DELSEY SA,NEW DELHI,Manufacture of other articles n.e.c.,14.84,0.22,2016,NaN,NaN
1,VETEA INDIA PRIVATE LIMITED,Australia,DECATHLON SA,NEW DELHI,Sports and recreation education,0.39,0.01,2016,NaN,NaN
2,MKA ENGINEERS AND EXPORTERS PVT LTD,Australia,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,Other business support service activities n.e.c.,15.00,0.22,2016,NaN,NaN
3,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,Australia,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,MUMBAI,"Manufacture of paints and varnishes, enamels o...",2.88,0.04,2016,NaN,NaN
4,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,Australia,ZF LENKSYSTEME GMBH,MUMBAI,"Manufacture of paints and varnishes, enamels o...",0.00,0.00,2016,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
15299,VAL PRODUCTS INDIA PVT COMPANY,U.S.A,CAMERON JAMES ALBORZIAN,MUMBAI,Manufacture of other dairy products n.e.c.,11.93,0.18,2016,NaN,NaN
15300,FOX CHANNELS (INDIA) PVT LTD,U.S.A,JAIYMSE MAYUMI HAFT,MUMBAI,Radio broadcasting,15.79,0.24,2016,NaN,NaN
15301,FOX CHANNELS (INDIA) PVT LTD,U.S.A,DHIRAJ SHAH,MUMBAI,Radio broadcasting,39.10,0.59,2016,NaN,NaN
15302,FOX CHANNELS (INDIA) PVT LTD,U.S.A,NAMRATA SHAH,MUMBAI,Radio broadcasting,42.09,0.63,2016,NaN,NaN


In [108]:
nalist6 = nalist6[['Company', 'State', 'District', 'Item', 'Foreign Collaborator', 'Country', 'Rupees', 'Dollars', 'Type', 'Year']]
nalist6

,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,PHARMACHOL CHEMICALS PRIVATE LIMITED,NaN,NEW DELHI,Manufacture of other articles n.e.c.,DELSEY SA,Australia,14.84,0.22,NaN,2016
1,VETEA INDIA PRIVATE LIMITED,NaN,NEW DELHI,Sports and recreation education,DECATHLON SA,Australia,0.39,0.01,NaN,2016
2,MKA ENGINEERS AND EXPORTERS PVT LTD,NaN,MUMBAI,Other business support service activities n.e.c.,ROBERT BOSCH AUTOMOTIVE STEERING GMBH,Australia,15.00,0.22,NaN,2016
3,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,NaN,MUMBAI,"Manufacture of paints and varnishes, enamels o...",ROBERT BOSCH AUTOMOTIVE STEERING GMBH,Australia,2.88,0.04,NaN,2016
4,CHROMAFLO TECHNOLOGIES INDIA PVT LTD,NaN,MUMBAI,"Manufacture of paints and varnishes, enamels o...",ZF LENKSYSTEME GMBH,Australia,0.00,0.00,NaN,2016
...,...,...,...,...,...,...,...,...,...,...
15299,VAL PRODUCTS INDIA PVT COMPANY,NaN,MUMBAI,Manufacture of other dairy products n.e.c.,CAMERON JAMES ALBORZIAN,U.S.A,11.93,0.18,NaN,2016
15300,FOX CHANNELS (INDIA) PVT LTD,NaN,MUMBAI,Radio broadcasting,JAIYMSE MAYUMI HAFT,U.S.A,15.79,0.24,NaN,2016
15301,FOX CHANNELS (INDIA) PVT LTD,NaN,MUMBAI,Radio broadcasting,DHIRAJ SHAH,U.S.A,39.10,0.59,NaN,2016
15302,FOX CHANNELS (INDIA) PVT LTD,NaN,MUMBAI,Radio broadcasting,NAMRATA SHAH,U.S.A,42.09,0.63,NaN,2016


In [109]:
nalist_c = pd.concat([nalist_c, nalist6])
nalist_c = nalist_c.reset_index(drop=True)
#I can see that I haven't cleaned up the countries very well. Write an algorithm to clean up the New Zealandin the Country column



In [110]:
%store nalist_c
nalist_c.head()


Stored 'nalist_c' (DataFrame)


,Company,State,District,Item,Foreign Collaborator,Country,Rupees,Dollars,Type,Year
0,ADANI INFRA (INDIA) LIMITED,GUJARAT,Gandhi Nagar,Construction and maintenance of power plants,HARMONIA TRADE AND INVESTMENT LIMITED,Mauritius,36471.99,501.29,Received in the Company,2021
1,INDUSIND BANK LIMITED,MAHARASHTRA,Pune,"Monetary intermediation of commercial\nbanks, ...",INDUSIND INTERNATIONAL HOLDINGS LIMITED,Mauritius,14858.61,204.12,Received in the Company,2021
2,PHONEPE PRIVATE LIMITED,DELHI,South Delhi,Activities auxiliary to financial service acti...,PHONEPE PRIVATE LIMITED,Singapore,11100.00,151.83,Received in the Company,2021
3,PEGATRON TECHNOLOGY INDIA PRIVATE LIMITED,TAMIL NADU,Kanchipuram,Manufacture of other communication equipments ...,PEGATRON CORPORATION,Taiwan,10448.96,143.54,Greenfield,2021
4,ZOMATO PRIVATE LIMITED,DELHI,South Delhi,Other information service activities n.e.c.,KORA HOLDINGS II C LLC,Cayman Islands,8452.47,116.12,Received in the Company,2021


## HELL CELL

In this cell, I cleaned up my countries just so that they could merge somewhat successfully. Many of them are islands, so i don't know what to do with that. 

In [1]:
map_df = nalist_c

#Time to clean up the countries in the Country column of map df. I need to make all the countries Match the countries in my geojson file
map_df.Country.unique()

# map_df.Country = map_df.Country.str.replace("Mauritiu s", "Mauritiana")
# map_df.Country = map_df.Country.str.replace("HongKon g", "Hong Kong")
# map_df.Country = map_df.Country.str.replace("Luxembo urg", "Luxembourg")
# map_df.Country = map_df.Country.str.replace("Singapor e", "Singapore")
# map_df.Country = map_df.Country.str.replace("U.K", "United Kingdom")
# map_df.Country = map_df.Country.str.replace("Scotland", #"United Kingdom")
# map_df.Country = map_df.Country.str.replace("U.A.E", "United Arab Emirates")
# map_df.Country = map_df.Country.str.replace("Korea(Nort h)", "North Korea")
# map_df.Country = map_df.Country.str.replace("MOZAMBIQ UE", "Mozambique")
# map_df.Country = map_df.Country.str.replace("MOZAMBI QUE", "Mozambique")
# map_df.Country = map_df.Country.str.replace("MOZAMBIQUE", "Mozambique")
# map_df.Country = map_df.Country.str.replace("Korea(North)", "North Korea")

#map_df.Country = map_df.Country.str.replace("BELORUSSI A", "Belarus")
map_df.Country = map_df.Country.str.replace("NewZealand", "New Zealand")
map_df.Country = map_df.Country.str.replace("", "New Zealand")


#MOZAMBIQ UE


#let's export map df to a csv file
map_df.to_csv('map_df.csv')

NameError: name 'nalist_c' is not defined

In [149]:
#Gruping the countries by total money in dollars and then sorting them by ascending order, then ouputtin the whole thing into a csv file

map_df.groupby('Country')['Dollars'].sum().sort_values(ascending=False).to_csv('map.csv')
len(map_df)

96809

In [49]:
# csv_path15 = "data/CSVs/2015"
# files = []
# for file in os.listdir(csv_path15):
#     if (file != ".DS_Store" and file != ".ipynb_checkpoints"):
#         files.append(os.path.join(csv_path15, file))

# dfs7 = []
# for file in files:
#     df = pd.read_csv(file)
#     if(df.iloc[0][1] != "Name of Indian Company"):
#         df = df.iloc[1:]
#         if(df.iloc[0][1] != "Name of Indian Company"):
#             df = df.iloc[1:]
#             if(df.iloc[0][1] != "Name of Indian Company"):
#                 df = df.iloc[1:]
#     df.columns = ["#", "Company", "Country", "Foreign Collaborator", "Regional Office", "Item", "Rupees", "Dollars", "NAN"]
#     df['#'].astype(str)
#     if(not isinstance(df.iloc[1][0], str)):
#         df = df.iloc[1:]
#     df.iloc[1, 0] = "01"
#     dfs7.append(df)
    

# merged7 = pd.concat(dfs7)[1:-1]
# merged7 = merged7.reset_index(drop=True)
# merged7.drop(merged7[merged7['Company'] == 'Name of Indian Company'].index, inplace = True)
# merged7.drop(merged7[merged7['Company'].isna()].index, inplace = True)
# merged7 = merged7.reset_index(drop=True)
# merged7

In [50]:
# nans7 = merged7[(merged7["#"].isna())]
# nans7.head(50)
# for i in range(len(merged7)) :
#     if(isinstance(merged7.loc[i, "#"], str) and len(merged7.loc[i, "#"]) == 1):
#         if (i != 0):
#             if(isinstance(merged7.loc[i - 1, "#"], str) and len(merged7.loc[i - 1, "#"]) == 3):
#                 print(merged7.loc[i])
#                 nans7 = nans7.append(merged7.loc[i])
# inc7 = nans7[nans7["Company"].notna()].index
# edited7 = merged7.drop(merged7[(merged7["#"].isna()) & (merged7['Company'].isna())].index)
# edited7 = edited7.drop(edited7[edited7['Company'].isna()].index)
# for num in inc7:
#     for label in ['Company']:
#         print(merged7.loc[num])
# #         if(str(merged7.loc[num, label]) != 'nan'):
# #             edited7.loc[num-1, label] =  merged7.loc[num - 1, label] + " " + merged7.loc[num, label]
# #     edited7.drop(labels=num, inplace=True)   
# # edited7 = edited7.reset_index(drop=True)
# edited7[edited7['Rupees'].isna()]
# for i in edited6.index:
#     edited6.loc[i, 'Regional Office'] = str(edited6.loc[i, 'Regional Office']).replace('\n',' ')
#     edited6.loc[i, 'Foreign Collaborator'] = str(edited5.loc[i, 'Foreign Collaborator']).replace('\n',' ')
#     edited6.loc[i, 'Company'] = str(edited6.loc[i, 'Company']).replace('\n',' ')
#     edited6.loc[i, 'Rupees'] = str(edited6.loc[i, 'Rupees']).replace('\n','')
#     edited6.loc[i, 'Dollars'] = str(edited6.loc[i, 'Dollars']).replace('\n','')